In [8]:
import pandas as pd
df = pd.read_csv('../data/bronze/european_visa_database/visa_practice_eu.csv')
df

,visaPracticeEuID,rcID,receivingCountryName,receivengCountryCode,scCityID,sendingCountryName,sendingCountryCode,sendingCityName,dYear,shortStayAppliedFor,...,issuedABCDVTL,issuedABCDDCVTL,appliedA,appliedB,appliedC,appliedABC,notIssuedA,notIssuedB,notIssuedC,notIssuedABC
0,31797,173,Belgium,BE,7,Algeria,DZ,Algiers,2005,5361.0,...,NaN,4372.0,NaN,NaN,5070.0,5075.0,NaN,NaN,NaN,1340.0
1,31798,173,Belgium,BE,11,Angola,AO,Luanda,2005,1910.0,...,NaN,1140.0,NaN,NaN,1699.0,1910.0,NaN,NaN,NaN,144.0
2,31799,173,Belgium,BE,13,Argentina,AR,Buenos Aires,2005,17.0,...,NaN,75.0,NaN,NaN,17.0,17.0,NaN,NaN,NaN,4.0
3,31800,173,Belgium,BE,21,Australia,AU,Canberra,2005,387.0,...,NaN,253.0,NaN,NaN,274.0,387.0,NaN,NaN,NaN,273.0
4,31801,173,Belgium,BE,281,Austria,AT,Vienna,2005,2.0,...,NaN,9.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19013,50810,185,Italy,IT,265,Zimbabwe,ZW,Harare,2012,669.0,...,NaN,NaN,0.0,NaN,665.0,665.0,0.0,NaN,0.0,NaN
19014,50811,190,Netherlands,NL,265,Zimbabwe,ZW,Harare,2012,391.0,...,NaN,NaN,NaN,NaN,391.0,391.0,NaN,NaN,NaN,NaN
19015,50812,170,Norway,NO,265,Zimbabwe,ZW,Harare,2012,596.0,...,NaN,NaN,0.0,NaN,524.0,524.0,0.0,NaN,40.0,40.0
19016,50813,192,Portugal,PT,265,Zimbabwe,ZW,Harare,2012,136.0,...,NaN,NaN,NaN,NaN,136.0,136.0,NaN,NaN,NaN,NaN


In [9]:
df.columns

Index(['visaPracticeEuID', 'rcID', 'receivingCountryName',
       'receivengCountryCode', 'scCityID', 'sendingCountryName',
       'sendingCountryCode', 'sendingCityName', 'dYear', 'shortStayAppliedFor',
       'shortStayIssued', 'shortStayRefused', 'shortStayRefusalRate',
       'issuedA_All', 'issuedA_Mev', 'issuedB', 'issuedC_All', 'issuedC_Mev',
       'issuedD', 'issuedDC', 'issuedVTL', 'issuedADS', 'issuedABC',
       'issuedABCVTL', 'issuedABCDVTL', 'issuedABCDDCVTL', 'appliedA',
       'appliedB', 'appliedC', 'appliedABC', 'notIssuedA', 'notIssuedB',
       'notIssuedC', 'notIssuedABC'],
      dtype='object')

Clean-up raw Schengen input file and save as silver statistics 

In [10]:
import numpy as np

column_mapping = {"receivingCountryName": "schengen_state"
                    ,"sendingCountryName": "origin_country"
                    ,"sendingCityName": "origin_consulate"
                    ,"appliedABC": "visas_applied"
                    ,"issuedABC": "visas_issued"
                    ,"notIssuedABC": "visas_not_issued"
                    ,"dYear": "year"}

df = df.rename(columns = column_mapping)
print('Renamed columns')

df["schengen_state"] = df["schengen_state"].str.strip()

df["origin_country"] = df["origin_country"].str.upper()
df["origin_consulate"] = df["origin_consulate"].str.upper()

df["visas_applied"].replace(np.nan, 0, inplace=True)
df["visas_issued"].replace(np.nan, 0, inplace=True)
df["visas_not_issued"].replace(np.nan, 0, inplace=True)
print('Replaced applied, issued and not issued null values with 0')

df["visas_issued"].loc[df["visas_issued"]<0] = 0
print('Replaced negative entries for visas issued with 0')

df["visa_refusal_rate"] = df["visas_not_issued"] / (df["visas_issued"] + df["visas_not_issued"])
print('Calculate refusal rate as the not issued share of the total issued and not issued')

df = df[["schengen_state", "origin_country","origin_consulate","visas_applied", "visas_issued", "visas_not_issued", "visa_refusal_rate", "year"]]
print('Selected columns')

df.to_csv('../data/silver/schengen-visa-evd.csv')

Renamed columns
Replaced applied, issued and not issued null values with 0
Replaced negative entries for visas issued with 0
Calculate refusal rate as the not issued share of the total issued and not issued
Selected columns


/var/folders/15/q71mgwyd7jndbh0bd__klkvw0000gn/T/ipykernel_8210/1930720416.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["visas_issued"].loc[df["visas_issued"]<0] = 0


Data quality checks

In [11]:
df[df["visas_applied"] != df["visas_issued"] + df["visas_not_issued"]]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
0,Belgium,ALGERIA,ALGIERS,5075.0,3738.0,1340.0,0.263883,2005
1,Belgium,ANGOLA,LUANDA,1910.0,1134.0,144.0,0.112676,2005
4,Belgium,AUSTRIA,VIENNA,2.0,3.0,0.0,0.000000,2005
8,Belgium,BULGARIA,SOFIA,203.0,120.0,57.0,0.322034,2005
9,Belgium,BURKINA FASO,OUAGADOUGOU,757.0,630.0,114.0,0.153226,2005
...,...,...,...,...,...,...,...,...
19002,Spain,YEMEN,SANA 'A,684.0,627.0,1.0,0.001592,2012
19003,Denmark,ZAMBIA,LUSAKA,489.0,256.0,6.0,0.022901,2012
19008,Netherlands,ZAMBIA,LUSAKA,251.0,243.0,7.0,0.028000,2012
19009,Sweden,ZAMBIA,LUSAKA,989.0,930.0,18.0,0.018987,2012


In [12]:
df[df["visa_refusal_rate"]>1]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year


In [13]:
df[df["visa_refusal_rate"].isna()]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
6,Belgium,BOLIVIA,LA PAZ,0.0,0.0,0.0,NaN,2005
51,Belgium,LUXEMBOURG,LUXEMBURG,0.0,0.0,0.0,NaN,2005
97,Denmark,AUSTRIA,VIENNA,1.0,0.0,0.0,NaN,2005
99,Denmark,BELGIUM,BRUSSELS,1.0,0.0,0.0,NaN,2005
115,Denmark,MACEDONIA,SKOPJE,1.0,0.0,0.0,NaN,2005
...,...,...,...,...,...,...,...,...
18357,Sweden,PORTUGAL,LISBON,3.0,0.0,0.0,NaN,2012
18450,Netherlands,RWANDA,KIGALI,1.0,0.0,0.0,NaN,2012
18517,Sweden,SINGAPORE,SINGAPORE,14.0,0.0,0.0,NaN,2012
18572,Sweden,SPAIN,MADRID,1.0,0.0,0.0,NaN,2012


In [14]:
df.describe()

,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
count,1.901800e+04,1.901800e+04,19018.000000,18393.000000,19018.000000
mean,5.372686e+03,5.063390e+03,339.514881,0.102824,2008.418656
std,2.568407e+04,2.488631e+04,1583.908476,0.159918,2.238062
min,0.000000e+00,0.000000e+00,0.000000,0.000000,2005.000000
25%,4.100000e+01,4.500000e+01,1.000000,0.003517,2006.000000
50%,4.570000e+02,4.210000e+02,15.000000,0.039216,2008.000000
75%,2.573250e+03,2.273750e+03,162.000000,0.132239,2010.000000
max,1.030969e+06,1.022444e+06,89499.000000,1.000000,2012.000000
